In [43]:
import sys
from datetime import datetime
import json
from sentence_transformers import SentenceTransformer
from langchain_community.llms import Ollama
import chromadb
import fitz
import regex as re
import uuid
# from chromadb.api import HttpClient
response_strings=[]

def fitz_extract_text(pdf_file_path):
    all_text = []
    doc = fitz.open(pdf_file_path)
    page_count = doc.page_count
    for page_num in range(page_count):
        page = doc[page_num]
        text = page.get_text()
        # text = page.get_text().replace('\n',' ')
        all_text.append(text)

    return all_text

def retrieve_and_generate(query, logfile):
    try:
        query_embedding = embedding_model.encode([query]).tolist()
        top_k = 1
        results = collectionIds.query(query_embedding, n_results=top_k)
        most_relevant_document = results['documents'][0]

        response = llm_model.generate(
            prompts=[f"Query: {query}\nDocument: {most_relevant_document}\nResponse:"]
        )

        return response
    except Exception as e:
        exc_type, exc_obj, tb = sys.exc_info()
        lineno = tb.tb_lineno
        ob = f'\nIn LLM Time: {datetime.now()} \nLINE.NO-{lineno} : {exc_obj}'
        with open(logfile, 'a', encoding='utf-8') as fp:
            fp.writelines(ob)


def processFile(all_text, logfile):
    print("processFile")
    try:
        embeddings = embedding_model.encode(all_text).tolist()
        # documents = [{"text": text, "embedding": embedding} for text, embedding in zip(all_text, embeddings)]
        documentsList = []
        embeddingList = []
        idsList = []
        n = 1
        for text, embedding in zip(all_text, embeddings):
            documentsList.append(text)
            embeddingList.append(embedding)
            idsList.append("ids" + str(n))
            n += 1
        collectionIds.add(documents=documentsList, embeddings=embeddingList, ids=idsList)

    except Exception as e:
        print("error=>processFile=>", e)
        exc_type, exc_obj, tb = sys.exc_info()
        lineno = tb.tb_lineno
        ob = f'\nIn LLM Time: {datetime.now()} \nLINE.NO-{lineno} : {exc_obj}'
        with open(logfile, 'a', encoding='utf-8') as fp:
            fp.writelines(ob)


def processPrompt(all_text, prompt, logfile):
    try:
        all_texts=[]
        text_to_pageNumber = {text: i for i, text in enumerate(all_text)}

        query_embedding = embedding_model.encode([prompt]).tolist()
        top_k = 1
        results = collectionIds.query(query_embedding, n_results=top_k)
        # print(results)
        most_relevant_document = results['documents'][0]
        page_number = (results['ids'][0][0]).split("ids")[1]
        # print(page_number)
        # page_number = text_to_pageNumber[most_relevant_document]

        response_text = llm_model.generate(
            prompts=[f"Query: {prompt}\nDocument: {most_relevant_document}\nResponse:"]
        )
        for generation_chunk in response_text.generations:
            for generation in generation_chunk:
                # all_texts.append(generation.text)
                pattern = r'{(?:[^{}]*|(?R))*}'
       
                json_text_match = re.search(pattern, generation.text, re.DOTALL)
                
                if json_text_match:
                    # print(json_text_match.group())
                    data_dict=json.loads(json_text_match.group().replace("'",'"'))
                    all_texts.append(data_dict)
        return all_texts
        # clean_data = str(response_text).replace('`', '')
        # if re.search(r'{.*}\\n', clean_data):
        #     data = re.search(r'{.*}\\n', clean_data).group(0)
        #     clean_data = data.replace('\n', '').replace('\\n', '').replace('\\', '')
        #     data_dict = json.loads(clean_data)
        #     final_response = {'Page': page_number + 1}
        #     for key, value in data_dict.items():
        #         final_response[key] = value
        #     return final_response
        # else:
        #     return "Data not found"
    except Exception as e:# Initialize ChromaDB client
        
        # Replace with your ChromaDB server URL
        print("error=>", e)
        exc_type, exc_obj, tb = sys.exc_info()
        lineno = tb.tb_lineno
        ob = f'\nIn LLM Time: {datetime.now()} \nLINE.NO-{lineno} : {exc_obj}'
        with open(logfile, 'a', encoding='utf-8') as fp:
            fp.writelines(ob)


In [44]:


logfile = 'log.txt'
pdf_file_path = r'pdf/PeterbiltIllinoisVJ.pdf'
all_text = fitz_extract_text(pdf_file_path)

# Initialize models
llm_model = Ollama(model="llama3:instruct", temperature=0)
# embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_model = Ollama(model='nomic-embed-text',temperature=0)

# Initialize ChromaDB client
chroma_client = chromadb.Client()

# Replace with your ChromaDB server URL


In [45]:
uuidName=str(uuid.uuid4())

In [46]:
# for name in [chromaName.name for chromaName in chroma_client.list_collections()]:
#     chroma_client.delete_collection(name)

In [47]:
fileName = 'PeterbiltIllinoisVJ3'
# Replace your collection name and settings accordingly
collection_tableName = "TableCollection"
collectionTable = chroma_client.get_or_create_collection(name=collection_tableName)

index = 0
if not fileName in [doc for doc in collectionTable.get()['documents']]:
    collectionTable.add(documents=[fileName], ids=[uuidName])

for doc in collectionTable.get()['documents']:
    if fileName == doc:
        # collectionTable.update(documents=[fileName],ids=[idsList for idsList in collectionTable.get(include=['documents'])]])
        print("TRUE")
        break
    index += 1
print(collectionTable.get()['ids'][index])

TRUE
ba4936e2-9933-4952-ad13-3325ea681722


In [48]:
print([chromaName.name for chromaName in chroma_client.list_collections()])
isCreate = collectionTable.get()['ids'][index] in [chromaName.name for chromaName in chroma_client.list_collections()]
print(isCreate)
if isCreate:
    collectionIds = chroma_client.get_collection(name=collectionTable.get()['ids'][index])
else:
    collectionIds = chroma_client.create_collection(name=collectionTable.get()['ids'][index])
# print(collectionTable.get())

['TableCollection', 'ba4936e2-9933-4952-ad13-3325ea681722']
True


In [49]:
if not isCreate:
    processFile(all_text, logfile)

In [50]:
from datetime import datetime
print('start',datetime.now())
prompt = "Extract the landlord address like {'landlordStreetAddress1':'','landlordStreetAddress2':'','City':'','State':'','Zipcode':''}"
result = processPrompt(all_text, prompt, logfile)
print('end',datetime.now())

start 2024-07-04 10:26:33.518695
error=> 'Ollama' object has no attribute 'encode'
end 2024-07-04 10:26:33.520667


In [42]:
result

In [39]:
import json
import chromadb
import uuid
import faiss
from datetime import datetime
from sentence_transformers import SentenceTransformer
config_data = json.loads(open("config.json", "r+").read())
RAG = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
chroma_client = chromadb.Client()
collection_tableName = config_data["tableName"]
collectionTable = chroma_client.get_or_create_collection(name=collection_tableName)
fileName="summary records sample.pdf"
all_text=json.loads(open(f"all_context{fileName.split('.pdf')[0]}.json","r+").read())

In [49]:
query="where is the vitals reports?"

In [40]:

# delete
get_collection=collectionTable.get()
print(get_collection)
doc_list=get_collection["documents"]
ids_list=get_collection["ids"]
for i in range(len(doc_list)):
    print(ids_list[i])
    collectionTable.delete(ids=[ids_list[i]])
print("after")

print(collectionTable.get())

{'ids': ['578a793c-89bf-4bba-a79b-0cdb1654d5a9'], 'embeddings': None, 'documents': ['summary records sample.pdf'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None]}
578a793c-89bf-4bba-a79b-0cdb1654d5a9
after
{'ids': [], 'embeddings': None, 'documents': [], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': []}


In [47]:
# text splitter
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
total_chunks=RecursiveCharacterTextSplitter(separators="\n",chunk_overlap=5)
all_chunks=[]
for context in all_text:
    chunk=total_chunks.split_text(re.sub("\s+"," ",context))
    all_chunks.extend(chunk)

In [50]:

# RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
top_k = int(len(all_chunks) * 0.2)
embeddings = RAG.encode(all_chunks, convert_to_numpy=True)
q_emb = RAG.encode([query], convert_to_numpy=True)
uuidName = str(uuid.uuid4())
print(collectionTable.get())
doc_list=collectionTable.get()["documents"]
ids_list=collectionTable.get()["ids"]
index_list= [i for i,data in enumerate(doc_list) if data==fileName]
if index_list:
    print(index_list)
    start=datetime.now()
    for index in index_list:
        isCreate = ids_list[index] in [chromaName.name for chromaName in chroma_client.list_collections()]
        print(isCreate)
        if isCreate:
            collectionIds = chroma_client.get_collection(name=ids_list[index])
        else:
            collectionIds = chroma_client.create_collection(name=ids_list[index])
        idsList = ["ids" + str(n) for n in range(len(all_text))]
        results = collectionIds.query(q_emb, n_results=top_k)
        print(results['documents'][0])
    end=datetime.now()
    print(f"total chromadb time:{end-start}")
else:
    start=datetime.now()
    index_ = faiss.IndexFlatL2(embeddings.shape[1])
    index_.add(embeddings)
    D, I = index_.search(q_emb, k=top_k)
    chunk_list = [all_text[i] for i in I[0]]
    embed_list = [embeddings[i] for i in I[0]]
    collectionTable.add(documents=[fileName], ids=[uuidName])
    index= [i for i,data in enumerate(collectionTable.get()["documents"]) if data==fileName][0]
    collectionIds = chroma_client.create_collection(name=collectionTable.get()['ids'][index])
    idsList = ["ids" + str(n) for n in range(len(chunk_list))]
    collectionIds.add(embeddings=embed_list, ids=idsList, documents=chunk_list)
    results = collectionIds.query(q_emb, n_results=top_k)
    end=datetime.now()
    print(f"total RAG time:{end-start}")
    print(results['documents'][0])

{'ids': ['3f278492-4924-46ce-b679-89dd98442f7b'], 'embeddings': None, 'documents': ['summary records sample.pdf'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None]}
[0]
True
['values, identifiers, dates, or findings across the entire medical record set. Each section has been\nintentionally expanded to provide comprehensive clinical context.\nSummary\nAppointment 24 documents an independent medical evaluation with individualized history,\ndiagnostics, assessment, and observations. The encounter concludes with conservative\nmanagement recommendations and plans for follow-up as needed.\n', 'values, identifiers, dates, or findings across the entire medical record set. Each section has been\nintentionally expanded to provide comprehensive clinical context.\nSummary\nAppointment 12 documents an independent medical evaluation with individualized history,\ndiagnostics, assessment, and observations. The encounter concludes with conservative\nmanagement rec

In [11]:

# # RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
# top_k = int(len(all_chunks) * 0.2)
# embeddings = RAG.encode(all_chunks, convert_to_numpy=True)
# q_emb = RAG.encode([query], convert_to_numpy=True)
# uuidName = str(uuid.uuid4())
# collectionTable.add(documents=[fileName], ids=[uuidName])
# index = 0
# print(collectionTable.get())
# doc_list=collectionTable.get()["documents"]
# ids_list=collectionTable.get()["ids"]
# for i in range(len(doc_list)):
#     if fileName == doc_list[i]:
#         collectionTable.update(documents=[fileName],ids=[ids_list[i]])
#         index=i
#         print("TRUE")
#         break
# print(index)
# isCreate = collectionTable.get()['ids'][index] in [chromaName.name for chromaName in
#                                                    chroma_client.list_collections()]
# if isCreate:
#     collectionIds = chroma_client.get_collection(name=collectionTable.get()['ids'][index])
# else:
#     collectionIds = chroma_client.create_collection(name=collectionTable.get()['ids'][index])

# idsList = ["ids" + str(n) for n in range(len(all_text))]
# collectionIds.add(embeddings=embeddings, ids=idsList, documents=all_text)
# results = collectionIds.query(q_emb, n_results=top_k)
# results['documents'][0]

{'ids': ['7113a4d7-8af4-49a1-a817-b35f94dddd74'], 'embeddings': None, 'documents': ['summary records sample.pdf'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None]}
TRUE
0


['values, identifiers, dates, or findings across the entire medical record set. Each section has been\nintentionally expanded to provide comprehensive clinical context.\nSummary\nAppointment 24 documents an independent medical evaluation with individualized history,\ndiagnostics, assessment, and observations. The encounter concludes with conservative\nmanagement recommendations and plans for follow-up as needed.\n',
 'values, identifiers, dates, or findings across the entire medical record set. Each section has been\nintentionally expanded to provide comprehensive clinical context.\nSummary\nAppointment 12 documents an independent medical evaluation with individualized history,\ndiagnostics, assessment, and observations. The encounter concludes with conservative\nmanagement recommendations and plans for follow-up as needed.\n',
 'values, identifiers, dates, or findings across the entire medical record set. Each section has been\nintentionally expanded to provide comprehensive clinical 

In [41]:
text="\n".join(all_text)
with open("ttxt.txt","w+")as f:
    f.write(text)